In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
import umap

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedShuffleSplit
# Converting labels to 1-Hot Vectors
from sklearn.preprocessing import OneHotEncoder
from mpl_toolkits.mplot3d import Axes3D


import sys
# sys.path.append("/Users/Work/Developer/interpretDL/interprettensor")
root_logdir = "./tf_logs"
datadir = "data/"

# To plot pretty figures
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

np.random.seed(seed=42) 
tf.__version__

Using TensorFlow backend.


'1.13.1'

In [2]:
# Helper Functions

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

######### Taken from sklearn #######
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots(figsize=[8,8])
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


def get1hot(y_train,y_test):
    from sklearn.preprocessing import OneHotEncoder

    enc = OneHotEncoder(categories="auto", sparse=False)
    y_train_1hot = enc.fit_transform([[label] for label in y_train]) # Since the function expects an array of "features" per sample
    y_test_1hot = enc.fit_transform([[label] for label in y_test])

    return y_train_1hot, y_test_1hot

def get_split(features, labels):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_index, test_index in split.split(features, labels):
        X_train = features[train_index]
        y_train = labels.iloc[train_index]
        X_test = features[test_index]
        y_test = labels.iloc[test_index]
        
        yield X_train, y_train, X_test, y_test

def plot_history(history):
    plt.close()
    fig, axs = plt.subplots(1, 2, figsize=(12,6))

    # Plot training & validation accuracy values
    axs[0].grid(True)
    axs[0].plot(history.history['binary_accuracy'])
    axs[0].plot(history.history['val_binary_accuracy'])
    axs[0].set(title='Model accuracy', ylabel='Accuracy', xlabel='Epoch')
    axs[0].legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    axs[1].grid(True)
    axs[1].plot(history.history['loss'])
    axs[1].plot(history.history['val_loss'])
    axs[1].set(title='Model loss',ylabel='Loss', xlabel='Epoch')
    axs[1].legend(['Train', 'Test'], loc='upper left')

    plt.show()


def remove_label(features, labels, label="MCI"):
    labels = pd.Series(fused_labels)
    non_samples = labels != label

    stripped_features = features[non_samples]
    stripped_labels = labels[non_samples]

    return stripped_features, stripped_labels

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

class AttributeRemover(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    
    def fit(self, X, y=None):
        return # Doesn't do anything
    
    def transform(self, X, y=None):
        return X.drop(columns=self.attribute_names)

class OverSampler(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, random_state=42):
        self.smote = SMOTE(random_state=random_state)
    
    def fit(self, X, y=None):
        return None
    
    def transform(self, X, y=None):
        return self.smote.fit_resample(X,y)

class AddNoise(BaseEstimator, TransformerMixin):
    """
    Returns a copy of matrix with attributes removed
    """
    def __init__(self, random_state=42):
        self.smote = SMOTE(random_state=random_state)
    
    def fit(self, X, y=None):
        return None
    
    def transform(self, X, y=None):
        return self.smote.fit_resample(X,y)
    
# Not used
train_pipeline = Pipeline([
                    ("smote", OverSampler()),
                    ("normalizer", StandardScaler()) ])

In [129]:
'''
Returns orginal samples, labels and modded_samples,modded_labels
'''
def modded_iris():

    from sklearn import datasets

    iris = datasets.load_iris()

    features = pd.DataFrame(iris["data"])
    target = pd.Series(iris["target"])
    flower_names = iris["target_names"]
    feature_names = iris["feature_names"]
    print(features.info())

    ### Get the first 2 flower samples

    setosa = target == 0
    versicolor = target == 1
    samples = features[setosa | versicolor]
    labels = target[setosa | versicolor]
    class_size = sum(versicolor)

    versicolor_samples = features[versicolor]
    versicolor_labels = target[versicolor]
    setosa_samples = features[setosa]

    ### Splitting *versicolor* into two sub classes

    versicolor_samples.describe()

    ## Constructing different noise sources
    gauss_noise = np.random.normal(loc=1,scale=0.25, size=[class_size//2,2])
    gauss_noise[gauss_noise < 0] = 0
    unif_noise = np.random.uniform(low=0,high=1)
    constant = 1


    split_size = class_size//2

    # Positive to first two features

    B1 = versicolor_samples.iloc[:split_size,:2] + gauss_noise
    B1 = np.concatenate((B1, versicolor_samples.iloc[:split_size,2:]), axis=1)
    B1_labels = versicolor_labels.iloc[:split_size]

    # Negative to last two features
    # gauss_noise = np.random.normal(loc=0.1,scale=0.1, size=[class_size//2,2])
    # gauss_noise[gauss_noise < 0] = 0
    # unif_noise = np.random.uniform(low=0,high=1)

    # B2 = versicolor_samples.iloc[split_size:,2:] + gauss_noise
    # B2 = np.concatenate((versicolor_samples.iloc[split_size:,2:],B2), axis=1)

    B2 = versicolor_samples.iloc[split_size:,:2] - gauss_noise
    B2 = np.concatenate((B2,versicolor_samples.iloc[split_size:,2:]), axis=1)
    B2_labels = versicolor_labels.iloc[split_size:] + 1

    # Combining the two fake "subclasses"
    noisy_samples = np.concatenate((B1, B2), axis=0)


    modded_samples = np.concatenate((setosa_samples,noisy_samples))
    modded_labels = labels.copy()
    modded_labels[class_size + split_size:] += 1

    return samples,labels,modded_samples, modded_labels


'''
Returns 8 gaussian blobs surrounding one center blob

       labels: Only center vs other labels (0,1) 
modded_labels: The labels for all 9 classes
'''
def make_blobs(class_size = 200):
    centers = [2*(x,y) for x in range(-1,2) for y in range(-1,2)]
    n_samples = [class_size//(len(centers)-1)]*len(centers)
    n_samples[len(centers)//2] = class_size

    X, y = make_blobs(n_samples=n_samples, centers=centers, n_features=2, cluster_std=0.1, shuffle=False, random_state=42)

    plt.close(1)
    df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
    fig, ax = plt.subplots()
    colors = {0:'red', 1:'blue'}
    df.plot(ax=ax,kind="scatter", x='x', y='y',c="label", cmap= "Paired")
    # plt.colorbar()
    plt.show()
    
    modded_labels = y.copy()
    modded_samples = df[["x","y"]].copy()
    labels = df["label"].copy()
    labels[labels != 4] = 0
    labels[labels == 4] = 1
    return modded_samples,labels, modded_labels

SyntaxError: invalid syntax (<ipython-input-129-5570591d73b2>, line 76)

## Using make_moons to generate sample data

In [5]:
from sklearn.datasets import make_circles
from sklearn.datasets.samples_generator import make_blobs
from matplotlib import pyplot
from pandas import DataFrame


In [135]:
# scatter plot, dots colored by class value

# generate 2d classification dataset
# X, y = make_circles(n_samples=100, noise=0)
class_size = 200
centers = [(0,0),(1,0),(-1,0),(0,1),(0,-1)]
n_samples = [class_size//(len(centers)-1)]*len(centers)
n_samples[0] = class_size

X, y = make_blobs(n_samples=n_samples, centers=centers, n_features=2, cluster_std=0.1, shuffle=False, random_state=42)

plt.close(1)
df = DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
fig, ax = plt.subplots(num=1)
colors = {0:'red', 1:'blue'}
df.plot(ax=ax,kind="scatter", x='x', y='y',c="label", cmap= "Paired")
# plt.colorbar()
plt.show()

modded_labels = y.copy()
modded_samples = df[["x","y"]].copy()
labels = df["label"].copy()
labels[labels > 0] = 1

FigureCanvasNbAgg()

In [59]:
def get_split_index(features, labels):
    features = np.array(features)
    # The train set will have equal amounts of each target class
    # Performing single split
    split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    return [[train_index, test_index] for train_index,test_index in split.split(features, labels)]

### Train a DNN on the modified dataset

In [98]:
# Get split returns a generator
# List comprehension is one way to evaluate a generator
# X_train, y_train, X_test, y_test = list(get_split(modded_samples, labels))[0]

train_index, test_index = get_split_index(modded_samples, modded_labels)[0]
features = modded_samples.copy()
X_train = features.iloc[train_index]
y_train = labels.iloc[train_index]
X_test = features.iloc[test_index]
y_test = labels.iloc[test_index]


print("Train Size:", X_train.shape)
print("Test Size:", y_test.shape)


hot_encoder = OneHotEncoder(categories="auto", sparse=False)
hot_encoder.fit(labels.values.reshape(-1,1)) # Since the function expects an array of "features" per sample
print("Categories:", hot_encoder.categories_)
# X_test, y_test.values

Train Size: (320, 2)
Test Size: (80,)
Categories: [array([0, 1])]


In [99]:
NUM_FEATURES = X_train.shape[1]
NUM_LABELS = len(hot_encoder.categories_[0])

In [100]:
def build_dnn(num_features, num_labels=3):

#     reset_graph()
    
    keras.backend.clear_session()

    nn = keras.models.Sequential()
    Dense = keras.layers.Dense
    
    # Using He initialization
    he_init = tf.keras.initializers.he_uniform()
    
    nn.add(Dense(units = 12, activation="elu", input_dim=num_features,
                kernel_initializer=he_init))
    nn.add(Dense(units = 12, activation="elu",
                kernel_initializer=he_init))
    nn.add(Dense(units=1, activation= "sigmoid",
                kernel_initializer=he_init))

#     BCE = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    nn.compile(loss="binary_crossentropy",
                  optimizer='sgd',
                  metrics=['binary_accuracy'])
    
    return nn

def train_model(model, X, y, X_test=[], y_test=[], epochs=30, batch_size=20, verbose=1, plot=True):
    
    ZScaler = StandardScaler().fit(X)
    
    X_train = X #ZScaler.transform(X)
#     X_test = ZScaler.transform(X_test)
    
    y_train = y.values
    y_test = y_test.values
    
#     lr_scheduler = keras.callbacks.LearningRateScheduler(exp_decay)
    callback_list = []
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size = batch_size,
                        validation_data=(X_test, y_test), callbacks=callback_list, verbose=verbose)
    
#     if plot: plot_history(history)
    
    return history, ZScaler


In [101]:
nn = build_dnn(NUM_FEATURES)
%time history, Zscaler = train_model(nn, X_train, y_train, X_test, y_test, epochs=50, batch_size=20)

Train on 320 samples, validate on 80 samples
Epoch 1/50
320/320 [==============================] - 0s 612us/step - loss: 0.7537 - binary_accuracy: 0.4719 - val_loss: 0.7339 - val_binary_accuracy: 0.4375
Epoch 2/50
320/320 [==============================] - 0s 51us/step - loss: 0.7118 - binary_accuracy: 0.4813 - val_loss: 0.6957 - val_binary_accuracy: 0.5875
Epoch 3/50
320/320 [==============================] - 0s 45us/step - loss: 0.6815 - binary_accuracy: 0.7438 - val_loss: 0.6674 - val_binary_accuracy: 0.8750
Epoch 4/50
320/320 [==============================] - 0s 51us/step - loss: 0.6589 - binary_accuracy: 0.8531 - val_loss: 0.6450 - val_binary_accuracy: 0.8625
Epoch 5/50
320/320 [==============================] - 0s 46us/step - loss: 0.6395 - binary_accuracy: 0.8188 - val_loss: 0.6257 - val_binary_accuracy: 0.8000
Epoch 6/50
320/320 [==============================] - 0s 50us/step - loss: 0.6227 - binary_accuracy: 0.7656 - val_loss: 0.6083 - val_binary_accuracy: 0.7875
Epoch 7/50
3

In [102]:
# Plotting results from history
plot_history(history)

FigureCanvasNbAgg()

In [103]:
nn.predict(X_test)
_labels = [np.float(x) for x in y_test]
nn.evaluate(X_test, y_test)

80/80 [==============================] - 0s 36us/step


[0.1255749136209488, 1.0]

In [104]:
# sess = tf.Session()
# predictions = nn.predict(Zscaler.transform(modded_samples))
# predictions = nn.predict(modded_samples)
# _labels = [np.float(x) for x in labels]

# sess.run(keras.metrics.binary_accuracy(_labels,predictions.flatten()))
# # predictions[0][0]
# # _labels
# # nn.evaluate(Zscaler.transform(modded_samples),labels.values)
# # nn.evaluate(modded_samples,labels.values)



# DONT FORGET TO RENORMALIZE WHEN EVALUATING SAMPLES

In [105]:
# from sklearn.model_selection import StratifiedKFold as KFold
# # keras.backend.clear_session()

# def getKF(X,y, n_splits=10):
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=42 ) #Default = 10

#     for train_index, test_index in kf.split(X,y):
#         X_train = X[train_index]
#         y_train = y.iloc[train_index]
#         X_test = X[test_index]
#         y_test = y.iloc[test_index]
        
#         yield X_train, y_train, X_test, y_test, test_index

# histories = []
# testing_indxs =[]
# predictions = []
# true_labels = []
# zoo = []
# for X_train, y_train, X_test, y_test, test_index in getKF(modded_samples, labels):
#     print(test_index)
#     dnn = build_dnn(NUM_FEATURES, NUM_LABELS)
#     history, ZScaler = train_model(dnn,X_train, y_train, X_test, y_test, verbose=0, plot=False, epochs=20, batch_size=10)
    
#     # Updating all information arrays
#     histories.append(history)
#     testing_indxs.append(test_index)
#     zoo.append(dnn)
    
#     y_pred_probs = dnn.predict(ZScaler.transform(X_test))
#     y_pred = np.argmax(y_pred_probs, axis=1)
#     y_true = np.argmax(hot_encoder.transform(y_test.values.reshape(-1,1)), axis=1)
    
#     predictions.extend(y_pred)
#     true_labels.extend(y_true)
    
#     print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(history.history["binary_accuracy"][-1], history.history["val_binary_accuracy"][-1]))

In [106]:
# # Num is the figure number and clear tells it to clear the figure if it already exists
# plt.close()
# fig, axs = plt.subplots(num="KF Eval",
#                         nrows=len(histories)//2, ncols=2,
#                         figsize=(10,10), sharex=True, sharey=True)
# axs=axs.flatten()
# dfs = []

# for i,history in enumerate(histories):
#     df = pd.DataFrame(history.history)
#     dfs.append(df)
# #     axs[i].grid(True)
#     df[["binary_accuracy","val_binary_accuracy"]].plot(ax=axs[i], grid=True)

In [107]:
# sess = tf.Session()
# idx = -1
# best_dnn = zoo[idx]

# predictions = best_dnn.predict(Zscaler.transform(modded_samples))
# _labels = [np.float(x) for x in labels]
# sess.run(keras.metrics.binary_accuracy(_labels,predictions.flatten()))

# best_dnn.evaluate(Zscaler.transform(modded_samples),labels.values)
# # NOT bestdnn.evaluate(modded_samples,labels.values)


## Performing SVM on Modded Samples

In [108]:
from sklearn.svm import SVC

svm_clf = Pipeline([
#     ("scaler", StandardScaler()),
    ("SVM", SVC(kernel= "rbf", gamma= "scale", C=1))
])

%time svm_clf.fit(X_train, y_train)
svm_clf.score(X_test, y_test)

CPU times: user 2.55 ms, sys: 777 µs, total: 3.33 ms
Wall time: 2.53 ms


1.0

## Performing LRP 

> ###  TODO: Work with only *test set* 

In [109]:
model = nn
scaled_samples = X_test #Zscaler.transform(modded_samples)
_labels = y_test
mod_labels = modded_labels[test_index]

predictions = model.predict(scaled_samples)
preds = np.array([np.round(x[0]) for x in predictions])
true_labels = [np.float(x) for x in _labels]

correct = preds == true_labels
# versicolor = true_labels == 1

print("SANITY CHECK")
loss_and_metrics = model.evaluate(scaled_samples[correct], _labels[correct])
print("Scores on test set: loss={:0.3f} accuracy={:.4f}".format(*loss_and_metrics))

SANITY CHECK
80/80 [==============================] - 0s 31us/step
Scores on test set: loss=0.126 accuracy=1.0000


In [110]:
_labels[correct].value_counts()

1    40
0    40
Name: label, dtype: int64

In [120]:
import innvestigate
import innvestigate.utils as iutils

def perform_analysis(model, analyzer, data, labels):
    analysis = analyzer.analyze(data)
    prediction = model.predict(data)
    
    df_anal = pd.DataFrame(analysis)
    
    return df_anal


# Stripping the softmax activation from the model
# model_wo_sm = iutils.keras.graph.model_wo_softmax(model)

# Creating an analyzer
lrp_E = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model=model, epsilon=1e-3)
lrp_Z = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPZPlus(model=model)
lrp_AB   = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha2Beta1(model=model)

# Getting all the samples that can be correctly predicted
test_idx = correct
all_samples = scaled_samples[test_idx]
all_labels = _labels[test_idx]


# perform_analysis(nn,gradient_analyzer,flowers,types)
all_lrp_AB = perform_analysis(model,lrp_AB, all_samples, all_labels)
all_lrp_E = perform_analysis(model,lrp_E, all_samples, all_labels)
all_lrp_Z = perform_analysis(model,lrp_Z, all_samples, all_labels)


In [112]:
all_lrp.describe()
# mod_labels[51:50]

,0,1
count,80.000000,80.000000
mean,0.599642,0.245042
std,1.720559,8.304279
min,-2.178570,-65.060898
25%,-0.044530,-0.436565
50%,0.094552,-0.012988
75%,0.435938,0.830085
max,5.298864,17.074886


In [113]:
# plt.close()
lrp_results = all_lrp
# # lrp_E_results = all_lrp_E
population = lrp_results.mean()
sorted_features = population.sort_values(ascending=False)

# sorted_features.plot(kind="bar")
# plt.xticks(rotation=65, fontsize="small")

# plt.show()

In [114]:
def get_relevant_cols(df, thresh = 1e-2):

    all_above_thresh = (df < thresh).all(0) #Check if all values in columns satisfy the criteria
    max_above_thresh = (df.max() < thresh)
    quantile_above_thresh = (df.quantile(0.8) < thresh)

    criteria = quantile_above_thresh
    irrelevant_cols = df.columns[criteria] 
    irrelevant_cols
    
    relevant_features_only = all_lrp.drop(columns = irrelevant_cols)
    
    return relevant_features_only



In [128]:
plt.close(4)
fig, axs = plt.subplots(2,2, figsize=(16,10))
_labels = mod_labels

df.plot(ax=axs[0][0],kind="scatter", x='x', y='y',c="label", cmap= "Paired", title="Original Distribution")
all_lrp_E.plot(ax=axs[0][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP E")

all_lrp_AB.plot(ax=axs[1][0],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP AB")
all_lrp_Z.plot(ax=axs[1][1],kind="scatter", x=0, y=1, c=_labels, cmap="Paired", s=20, alpha=0.75, title="LRP Z")

plt.tight_layout()
plt.show()

FigureCanvasNbAgg()

## Plotting 2D LRP

In [88]:
_labels = mod_labels
plt.close()
fig, ax = plt.subplots(figsize=(10,8))
all_lrp_E.plot(ax=axs[1],kind="scatter", x=0, y=1, c=_labels, cmap="Set1", s=20, alpha=0.5)

FigureCanvasNbAgg()

In [89]:
plt.close()
fig, ax = plt.subplots(figsize=(12,10))
pos_only = all_lrp.copy()
pos_only[pos_only < 0] = 0
pos_only.plot(ax=ax,kind="scatter", x=0, y=1, c=_labels, cmap="Spectral", s=20, alpha=0.5)

FigureCanvasNbAgg()

## Plotting UMAP projections

In [91]:
def plot_2d(X,labels, name="1"):
    plt.close()
    fig = plt.figure(figsize=(10,8))
    plt.scatter(x=X[:,0], y=X[:,1], s= 10, cmap="Paired", c=labels)
    plt.colorbar()

    
pos_only = all_lrp.copy()
pos_only[pos_only < 0] = 0

_labels = mod_labels[correct]


# pos_only = lrp_results.copy()
plt.close()
# pos_only[pos_only < 0] = 0


reducer_2d = umap.UMAP(random_state=42,
                    n_components = 2,
                    n_neighbors=25,
                    min_dist=0)


embedding_2d = reducer_2d.fit_transform(pos_only)


plot_2d(embedding_2d, _labels, name="Alpha")

# %time reducer_3d = umap.UMAP(random_state=42, n_components = 3, n_neighbors=25, min_dist=0)
# embedding_3d = reducer_3d.fit_transform(pos_only)
# fig = plt.figure(figsize=(12,10))
# ax = fig.add_subplot(111, projection='3d')
# _im = ax.scatter(embedding_3d[:,0], embedding_3d[:,1], embedding_3d[:,2], c=_labels, s=20, cmap= "Paired")
# plt.colorbar(_im)

plt.show()


FigureCanvasNbAgg()

## LRP-Epsilon

In [ ]:
pos_only = all_lrp_E.copy()
pos_only[pos_only < 0] = 0

_labels = modded_labels[correct]


plt.close()

reducer_3d = umap.UMAP(random_state=42,
                    n_components = 3,
                    n_neighbors=25,
                    min_dist=0)

reducer_2d = umap.UMAP(random_state=42,
                    n_components = 2,
                    n_neighbors=15,
                    min_dist=0)

embedding_3d = reducer_3d.fit_transform(pos_only)
embedding_2d = reducer_2d.fit_transform(pos_only)

plot_2d(embedding_2d, labels, name="Alpha")

fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')
_im = ax.scatter(embedding_3d[:,0], embedding_3d[:,1], embedding_3d[:,2], c=modded_labels, s=40)
plt.colorbar(_im)

plt.show()


In [ ]:
df = pd.DataFrame(pos_only)

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')
_im = ax.scatter(df[1], df[1], df[2], c=labels, s=40)
plt.colorbar(_im)

plt.show()

In [ ]:


plot_2d(df[[2,3]].values, labels)